In [1]:
import sys
# 移动默认路径
sys.path.append("../")

In [2]:
import os
import re
import jieba
import pandas as pd

In [3]:
from utils.path_config import *

### 导入csv文件

In [4]:
# 通过正则表达式去除停用词
def segment_line(line):
    line = re.sub(
            "[a-zA-Z0-9]|[\s+\-\|\!\/\[\]\{\}_,.$%^*(+\"\')]+|[:：+——()?【】《》“”！，。？、~@#￥%……&*（）]+|题目", '',line)
    tokens = jieba.cut(line, cut_all=False)
    return " ".join(tokens)

In [5]:
def get_segment(subject_section, subject_path):
    iter_ = 0
    for label, path in zip(subject_section, subject_path):
        df = pd.read_csv(path)
        df['item'] = df['item'].apply(lambda x:segment_line(x))
        df['label'] = label
        # 第一次循环初始化dataset_df
        if iter_ == 0:
            dataset_df = df
            iter_ += 1
        else:
            dataset_df=pd.concat([dataset_df,df])
    return dataset_df

In [6]:
%%time
history_df = get_segment(history_section, history_path_all)
politics_df = get_segment(politics_section, politics_path_all)
geography_df = get_segment(geography_section, geography_path_all)
biological_df = get_segment(biological_section, biological_path_all)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.917 seconds.
Prefix dict has been built successfully.


CPU times: user 35.8 s, sys: 46.6 ms, total: 35.8 s
Wall time: 36.1 s


In [7]:
dataset_df = pd.concat([history_df, politics_df, geography_df, biological_df])

In [8]:
dataset_df.head()

,web-scraper-order,web-scraper-start-url,item,label
0,1566523436-2497,https://study.baidu.com/tiku,据 左传 记载 春秋 后期 鲁国 大夫 季孙氏 的 家臣 阳虎 独掌 权柄 后 标榜 要 替...,古代史
1,1566523436-2506,https://study.baidu.com/tiku,秦始皇 统一 六国后 创制 了 一套 御玺 如 任命 国家 官员 则 封印 皇帝 之玺 ； ...,古代史
2,1566523436-2153,https://study.baidu.com/tiku,北宋 加强 中央集权 的 主要 措施 有 ① 把 主要 将领 的 兵权 收归 中央 ② 派 ...,古代史
3,1566523436-2328,https://study.baidu.com/tiku,商朝人 崇信 各种 鬼神 把 占卜 祭祀 作为 与 神灵 沟通 的 手段 负责 通神 事务 ...,古代史
4,1566523436-1914,https://study.baidu.com/tiku,公元 年 北宋 政府 在 江淮地区 设置 了 包括 盐业 管理 以及 控制 对 茶叶 销售 ...,古代史


In [9]:
dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26243 entries, 0 to 1039
Data columns (total 4 columns):
web-scraper-order        26243 non-null object
web-scraper-start-url    26243 non-null object
item                     26243 non-null object
label                    26243 non-null object
dtypes: object(4)
memory usage: 1.0+ MB


In [10]:
corpus=dataset_df['item']

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer

### TDIDF

In [12]:
%%time
# 设置2500个特征值，忽略词频小于5的词。
vectorizer = TfidfVectorizer(max_features=2500,min_df=5)
X = vectorizer.fit_transform(corpus)

CPU times: user 2.67 s, sys: 19.7 ms, total: 2.69 s
Wall time: 2.74 s


In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test=train_test_split(X.toarray(),dataset_df['label'],test_size=0.2,random_state=42)

## sklearn
[naive bayes](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.naive_bayes)

In [15]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,ComplementNB,BernoulliNB
from sklearn.metrics import classification_report

# Naive Bayes

## 高斯朴素贝叶斯

[`GaussianNB`](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB "sklearn.naive_bayes.GaussianNB") 实现了运用于分类的高斯朴素贝叶斯算法。特征的可能性(即概率)假设为高斯分布:

$$
P(x_i \mid y) = \frac{1}{\sqrt{2\pi\sigma^2_y}} \exp\left(-\frac{(x_i - \mu_y)^2}{2\sigma^2_y}\right)
$$

参数 $\sigma_y$ 和 $\mu_y$ 使用最大似然法估计。

```py
>>> from sklearn import datasets
>>> iris = datasets.load_iris()
>>> from sklearn.naive_bayes import GaussianNB
>>> gnb = GaussianNB()
>>> y_pred = gnb.fit(iris.data, iris.target).predict(iris.data)
>>> print("Number of mislabeled points out of a total %d points : %d"
...       % (iris.data.shape[0],(iris.target != y_pred).sum()))
Number of mislabeled points out of a total 150 points : 6

```

In [24]:
%%time
# 高斯朴素贝叶斯
clf = GaussianNB()
clf.fit(X_train, y_train)
Gauss_pred = clf.predict(X_test)

CPU times: user 1.24 s, sys: 461 ms, total: 1.71 s
Wall time: 1.71 s


## 多项分布朴素贝叶斯

[`MultinomialNB`](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB "sklearn.naive_bayes.MultinomialNB") 实现了服从多项分布数据的朴素贝叶斯算法，也是用于文本分类(这个领域中数据往往以词向量表示，尽管在实践中 tf-idf 向量在预测时表现良好)的两大经典朴素贝叶斯算法之一。 分布参数由每类 $y$ 的 $\theta_y = (\theta_{y1},\ldots,\theta_{yn})$ 向量决定， 式中 $n$ 是特征的数量(对于文本分类，是词汇量的大小) $\theta_{yi}$ 是样本中属于类 $y$ 中特征 $i$ 概率 $P(x_i \mid y)$。

参数 $\theta_y$ 使用平滑过的最大似然估计法来估计，即相对频率计数:

$$
\hat{\theta}_{yi} = \frac{ N_{yi} + \alpha}{N_y + \alpha n}
$$

式中$N_{yi} = \sum_{x \in T} x_i$是 训练集T中特征i在类$y$中出现的次数，$N_{y} = \sum_{i=1}^{|T|} N_{yi}$ 是类 $y$中出现所有特征的计数总和。

先验平滑因子 $\alpha \ge 0$ 为在学习样本中没有出现的特征而设计，以防在将来的计算中出现0概率输出。 把 $\alpha = 1$ 被称为拉普拉斯平滑(Lapalce smoothing)，而 $\alpha \lt\ 1$被称为Lidstone平滑方法(Lidstone smoothing)。


In [25]:
%%time
# 多项式朴素贝叶斯
clf = MultinomialNB()
clf.fit(X_train, y_train)
Multi_pred = clf.predict(X_test)

CPU times: user 330 ms, sys: 48.9 ms, total: 379 ms
Wall time: 157 ms


## 补充朴素贝叶斯

[`ComplementNB`](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.ComplementNB.html#sklearn.naive_bayes.ComplementNB)实现了补充朴素贝叶斯(CNB)算法。CNB是标准多项式朴素贝叶斯(MNB)算法的一种改进，特别适用于不平衡数据集。具体来说，CNB使用来自每个类的补数的统计数据来计算模型的权重。CNB的发明者的研究表明，CNB的参数估计比MNB的参数估计更稳定。此外，CNB在文本分类任务上通常比MNB表现得更好(通常有相当大的优势)。计算权重的步骤如下:


$$
\begin{align}\begin{aligned}\hat{\theta}_{ci} = \frac{\alpha_i + \sum_{j:y_j \neq c} d_{ij}}{\alpha + \sum_{j:y_j \neq c} \sum_{k} d_{kj}}\\w_{ci} = \log \hat{\theta}_{ci}\\w_{ci} = \frac{w_{ci}}{\sum_{j} |w_{cj}|}\end{aligned}\end{align}
$$
其中对不在类c中的所有记录j求和，d<sub>ij</sub>可以是文档ｊ中词语i的计数或tf-idf值，α<sub>i</sub>是就像MNB中一样的平滑超参数，同时$\alpha = \sum_{i} \alpha_i$。第二个归一化解决了长记录主导MNB参数估计的问题。分类规则为:

$\hat{c} = \arg\min_c \sum_{i} t_i w_{ci}$
即将记录分配给补充匹配度最低的类。
                 

In [26]:
%%time
# 补充朴素贝叶斯
clf = ComplementNB()
clf.fit(X_train, y_train)
Comp_pred = clf.predict(X_test)

CPU times: user 367 ms, sys: 418 ms, total: 785 ms
Wall time: 243 ms


## 伯努利朴素贝叶斯

[`BernoulliNB`](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html#sklearn.naive_bayes.BernoulliNB "sklearn.naive_bayes.BernoulliNB") 实现了用于多重伯努利分布数据的朴素贝叶斯训练和分类算法，即有多个特征，但每个特征 都假设是一个二元 (Bernoulli, boolean) 变量。 因此，这类算法要求样本以二元值特征向量表示；如果样本含有其他类型的数据， 一个 `BernoulliNB` 实例会将其二值化(取决于 `binarize` 参数)。

伯努利朴素贝叶斯的决策规则基于:

$$
P(x_i \mid y) = P(i \mid y) x_i + (1 - P(i \mid y)) (1 - x_i)
$$

与多项分布朴素贝叶斯的规则不同 伯努利朴素贝叶斯明确地惩罚类 $y$ 中没有出现作为预测因子的特征 $i$ ，而多项分布分布朴素贝叶斯只是简单地忽略没出现的特征。

在文本分类的例子中，统计词语是否出现的向量(word occurrence vectors)(而非统计词语出现次数的向量(word count vectors))可以用于训练和使用这个分类器。 `BernoulliNB` 可能在一些数据集上表现得更好，特别是那些更短的文档。 如果时间允许，建议对两个模型都进行评估。

> **参考资料**:
>*   C.D. Manning, P. Raghavan and H. Schütze (2008). Introduction to Information Retrieval. Cambridge University Press, pp. 234-265.
>*   A. McCallum and K. Nigam (1998). [A comparison of event models for Naive Bayes text classification.](http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.46.1529) Proc. AAAI/ICML-98 Workshop on Learning for Text Categorization, pp. 41-48.
>*   V. Metsis, I. Androutsopoulos and G. Paliouras (2006). [Spam filtering with Naive Bayes – Which Naive Bayes?](http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.61.5542) 3rd Conf. on Email and Anti-Spam (CEAS).

In [27]:
%%time
# 伯努利朴素贝叶斯
clf = BernoulliNB()
clf.fit(X_train, y_train)
Bern_pred = clf.predict(X_test)

CPU times: user 681 ms, sys: 693 ms, total: 1.37 s
Wall time: 566 ms


In [28]:
print("高斯朴素贝叶斯结果：")
print(classification_report(Gauss_pred, y_test))
print("多项式朴素贝叶斯结果：")
print(classification_report(Multi_pred, y_test))
print("补充朴素贝叶斯结果：")
print(classification_report(Comp_pred, y_test))
print("伯努利朴素贝叶斯结果：")
print(classification_report(Bern_pred, y_test))



高斯朴素贝叶斯结果：
              precision    recall  f1-score   support

       人口与城市       0.84      0.77      0.80       356
   公民道德与伦理常识       0.82      0.83      0.82       370
       分子与细胞       0.89      0.92      0.90       526
     区域可持续发展       0.25      0.38      0.30        16
         古代史       0.82      0.58      0.68       275
       地球与地图       0.57      0.27      0.36       184
      宇宙中的地球       0.83      0.98      0.90       621
        时事政治       0.17      1.00      0.29         2
         现代史       0.23      0.49      0.31       223
    现代生物技术专题       0.69      0.33      0.44       376
   生产活动与地域联系       0.72      0.77      0.74       261
    生活中的法律常识       0.39      0.89      0.54        18
      生物技术实践       0.33      0.55      0.41       215
     生物科学与社会       0.10      0.67      0.18       125
      科学思维常识       0.33      0.74      0.46        19
    科学社会主义常识       0.75      0.71      0.73       120
       稳态与环境       0.86      0.20      0.33       857
       经济学常识    

/home/abner/anaconda3/envs/nlp02/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
